<a href="https://colab.research.google.com/github/bhavya6701/comp473-project/blob/main/comp473_notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Implementation of Artistic Style Transfer Using Convolutional Neural Networks
**Authors:** Shibin Koshy [40295019], Ruturajsinh Vihol [40154693], Bhavya Manjibhai Ruparelia [40164863]

## Imports

In [ ]:
import os
import json
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import torch
import torch.optim as optim
from torchvision import transforms, models
from tqdm import tqdm
import warnings


In [ ]:
# Filter out warnings
warnings.filterwarnings('ignore')

# Set the home directory
HOME = os.getcwd()

# Set the device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

## Load Models

**VGG-16**: A convolutional neural network with 16 layers, popular for style transfer due to its ability to capture detailed hierarchical features across layers. Pre-trained on ImageNet, it offers a balance between depth and computational efficiency.

**VGG-19**: An extended version of VGG-16 with 19 layers, providing deeper feature representations. This model can capture more complex details, enhancing style extraction for artistic image synthesis.

<!-- **ResNet-50**: A 50-layer residual network with skip connections, which helps retain both high- and low-level features. This architecture is well-suited for extracting intricate textures and patterns in style transfer tasks. -->

In [ ]:
# Load pre-trained models from torchvision
model_dict = {}

# VGG-16
model_dict["vgg-16"] = models.vgg16(weights=models.VGG16_Weights.DEFAULT).features

# VGG-19
model_dict["vgg-19"] = models.vgg19(weights=models.VGG19_Weights.DEFAULT).features

# For each model, freeze all the parameters
for model in model_dict.values():
    for param in model.parameters():
        param.requires_grad = False

    # Move the model to the specified device (GPU or CPU)
    model.to(device)

## Load Images

In [ ]:
def load_image(img_path, max_size=400, shape=None):
    # Load in and transform an image, resize it, and convert it to a PyTorch tensor
    image = Image.open(img_path).convert("RGB")

    # Set target size based on max size or shape
    if shape is not None:
        target_size = shape
    else:
        target_size = min(max(image.size), max_size)

    # Define the transformation pipeline
    in_transform = transforms.Compose(
        [
            transforms.Resize(target_size),
            transforms.ToTensor(),
            transforms.Normalize(
                mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)
            ),
        ]
    )

    image = in_transform(image)[:3, :, :].unsqueeze(0)

    return image


content = load_image(HOME + "/data/input_images/content8.jpeg").to(device)
style = load_image(HOME + "/data/input_images/style8.png", shape=content.shape[-2:]).to(
    device
)

In [ ]:
# Convert the images to numpy arrays
def tensor_to_image(tensor):
    # Invert normalization by reversing the mean and std
    denormalize = transforms.Normalize(
        mean=[-0.485 / 0.229, -0.456 / 0.224, -0.406 / 0.225],
        std=[1 / 0.229, 1 / 0.224, 1 / 0.225],
    )

    # Apply denormalization, convert to numpy, squeeze and transpose (C x H x W -> H x W x C)
    image = denormalize(tensor).cpu().numpy().squeeze().transpose(1, 2, 0)

    # Clip values to stay within the [0, 1] range
    return np.clip(image, 0, 1)


# Display the images
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(20, 10))

ax1.title.set_text("Content Image")
ax1.imshow(tensor_to_image(content), label="Content")

ax2.title.set_text("Style Image")
ax2.imshow(tensor_to_image(style), label="Style")

# Remove the axes
ax1.axis("off")
ax2.axis("off")

plt.show()

## Feature Extraction and Gram Matrix

In [ ]:
def extract_features(image, model, layers):
    features = {}
    x = image
    
    # Iterate through the model layers
    for name, layer in model._modules.items():
        # Apply each layer to the image and store the result
        x = layer(x)
        if name in layers:
            features[layers[name]] = x

    return features


def gram_matrix(tensor):
    # Get the batch size, depth, height, and width of the tensor
    _, d, h, w = tensor.size()
    
    # Reshape the tensor to have the shape (depth, height * width)
    tensor = tensor.view(d, h * w)
    
    # Compute the Gram matrix
    gram = torch.mm(tensor, tensor.t())
    
    return gram

## Loss Functions and Optimization

In [ ]:
# Load the JSON file
with open("model_config.json", "r") as json_file:
    data = json.load(json_file)

# Extracting data into variables
layers = data["layers"]
model_style_weights = data["style_weights"]

## Training and Evaluation of Style Transfer Models

In [ ]:
# Function to calculate the content loss
def calculate_content_loss(target_features, content_features, layer):
    return torch.mean((target_features[layer] - content_features[layer].detach()) ** 2)


# Function to calculate the style loss
def calculate_style_loss(target_features, style_features, model_style_weights):
    style_loss = 0
    for layer, weight in model_style_weights.items():
        target_feature = target_features[layer]
        _, d, h, w = target_feature.shape
        target_gram = gram_matrix(target_feature)
        style_gram = gram_matrix(style_features[layer])
        layer_style_loss = weight * (torch.mean((target_gram - style_gram) ** 2))
        style_loss += layer_style_loss / (d * h * w)
    return style_loss


# Function to show the images and total loss
def plot_images_graph(images, total_losses, steps, checkpoints):
    # Create a plot with 2 rows and 6 columns
    fig, axes = plt.subplots(3, 4, figsize=(18, 10))

    # Flatten axes for easy iteration
    axes = axes.flatten()
    for i, image in enumerate(images):
        title = (
            f"Iteration {i * (steps // checkpoints):,}"
            if i > 0 and i < len(images) - 1
            else "Initial Image (Noise)"
            if i == 0
            else "Final Image"
        )
        axes[i].imshow(image)
        axes[i].axis("off")
        axes[i].set_title(title)

    # Show the plot
    plt.tight_layout()
    plt.show()

    # Plot the total loss values
    plt.figure(figsize=(10, 5))
    plt.plot(total_losses, label="Total Loss")
    plt.xlabel("Iteration")
    plt.ylabel("Total Loss")
    plt.title("Total Loss vs Iteration")
    plt.legend()
    plt.show()

In [ ]:
# Compute the content and style featuref for the content and style images using the VGG-16 and VGG-19 models
content_features = {
    "vgg-16": extract_features(
        content, model_dict["vgg-16"], layers["vgg-16"]
    ),
    "vgg-19": extract_features(
        content, model_dict["vgg-19"], layers["vgg-19"]
    )
}
style_features = {
    "vgg-16": extract_features(style, model_dict["vgg-16"], layers["vgg-16"]),
    "vgg-19": extract_features(style, model_dict["vgg-19"], layers["vgg-19"])
}

style_grams = {
    "vgg-16": {layer: gram_matrix(style_features["vgg-16"][layer]) for layer in style_features["vgg-16"]},
    "vgg-19": {layer: gram_matrix(style_features["vgg-19"][layer]) for layer in style_features["vgg-19"]}
}

In [ ]:
#  Function for the style transfer algorithm
def style_transfer_algorithm(
    model_name,
    content_loss_layer,
    iterations=1000,
    lr=0.01,
    checkpoints=10,
    alpha=1,
    beta=1e6,
):
    model = model_dict[model_name]
    model_content_features = content_features[model_name]
    model_style_features = style_features[model_name]

    # Ensure that the content tensor is moved to the correct device and requires gradient
    target = content.clone().requires_grad_(True).to(device)

    # Define the optimizer
    optimizer = optim.Adam([target], lr=lr)

    # Create a list to store the images and the total loss values
    images = [tensor_to_image(target.detach())]
    total_losses = []

    # Iterate through the steps
    for step in tqdm(range(1, iterations + 1)):
        # Extract the features from the target image
        target_features = extract_features(target, model, layers[model_name])

        # Calculate the content and style loss
        content_loss = calculate_content_loss(
            target_features, model_content_features, content_loss_layer
        )
        style_loss = calculate_style_loss(
            target_features, model_style_features, model_style_weights
        )

        # Compute the total loss
        total_loss = alpha * content_loss + beta * style_loss

        # Update the target image, zero the gradients, backpropagate, and step the optimizer
        optimizer.zero_grad()
        total_loss.backward()
        optimizer.step()

        # Append the total loss to the list
        total_losses.append(total_loss.item())

        # Save the images at the specified checkpoints
        if step % (iterations // checkpoints) == 0:
            images.append(tensor_to_image(target.detach()))

    print(
        f"Step {step}/{iterations} - Total loss: {total_loss.item():.16f}, "
        f"Content loss: {content_loss.item():.16f}, Style loss: {style_loss.item():.16f}"
    )

    # Include the final image
    images.append(tensor_to_image(target.detach()))

    return images, total_losses

In [ ]:
# Define hyperparameters and optimization settings
params = data["hyperparameters"]

iterations = params["iterations"]
checkpoints = params["checkpoints"]
content_weight = params["content_weight"]
style_weight = params["style_weight"]
lr = params["lr"]

content_loss_layer = data["content_layer"]

## VGG-16 Style Transfer

In [ ]:
# Call the style transfer function for the VGG-16 model
saved_images, total_losses = style_transfer_algorithm(
    "vgg-16", content_loss_layer, iterations, lr, checkpoints, 1, 1e6
)

vgg16_output_image = saved_images[-1]

# Create a plot with the images and total loss values
plot_images_graph(saved_images, total_losses, iterations, checkpoints)

## VGG-19 Style Transfer

In [ ]:
# Call the style transfer function for the VGG-19 model
saved_images, total_losses = style_transfer_algorithm(
    "vgg-19", content_loss_layer, iterations, lr, checkpoints, content_weight, style_weight
)

vgg19_output_image = saved_images[-1]

# Create a plot with the images and total loss values
plot_images_graph(saved_images, total_losses, iterations, checkpoints)

## Initial Content vs Final Style Transfer Image

In [ ]:
# Plot the content, style and final images from VGG-16 and VGG-19
fig, axes = plt.subplots(2, 2, figsize=(20, 10))

axes[0][0].imshow(tensor_to_image(content))
axes[0][0].axis("off")
axes[0][0].set_title("Content Image")

axes[0][1].imshow(tensor_to_image(style))
axes[0][1].axis("off")
axes[0][1].set_title("Style Image")

axes[1][0].imshow(vgg16_output_image)
axes[1][0].axis("off")
axes[1][0].set_title("VGG-16 Final Image")

axes[1][1].imshow(vgg19_output_image)
axes[1][1].axis("off")
axes[1][1].set_title("VGG-19 Final Image")

plt.show()

## Content Layer Comparison

In [ ]:
compare_layers = ["conv1_1", "conv2_1", "conv3_1", "conv4_1", "conv5_1"]
layer_images = []

# Iterate through the layers
for layer in compare_layers: 
    print(f"Running style transfer for layer: {layer}")
    
    # Call the style transfer function for the VGG-19 model
    saved_images, total_losses = style_transfer_algorithm(
        "vgg-19", layer, iterations, lr, checkpoints, content_weight, style_weight
    )

    # Append the final image to the list
    layer_images.append(saved_images[-2])

# Create a plot with all the images obtained using different style_weight values
fig, axes = plt.subplots(1, 5, figsize=(20, 4))

for i, image in enumerate(layer_images):
    axes[i].imshow(image)
    axes[i].axis("off")
    axes[i].set_title(f"Layer {compare_layers[i]}")

plt.tight_layout()
plt.show()


## Content Weight to Style Weight Ratio Analysis

In [ ]:
content_weights = [1e-5, 1e-4, 1e-3, 1e-2, 1e-1]
output_images = []

# Call the style transfer function for the VGG-19 model
for cw in content_weights:
    print(f"Running style transfer algorithm with content weight = {cw}")
    saved_images, total_losses = style_transfer_algorithm(
        "vgg-19",
        "conv4_2",
        500,
        lr,
        checkpoints,
        cw,
        style_weight
    )
    
    output_images.append(saved_images[-2])

# Create a plot with all the images obtained using different style weight values
fig, axes = plt.subplots(1, 5, figsize=(20, 4))

for i, image in enumerate(output_images):
    axes[i].imshow(image)
    axes[i].axis("off")
    axes[i].set_title(f"Content weight = {content_weights[i]}")
    
plt.tight_layout()
plt.show()